Write a program to solve a Sudoku puzzle by filling the empty cells.

A sudoku solution must satisfy all of the following rules:

1) Each of the digits 1-9 must occur exactly once in each row.
2) Each of the digits 1-9 must occur exactly once in each column.
3) Each of the digits 1-9 must occur exactly once in each of the 9 3x3 sub-boxes of the grid.

The '.' character indicates empty cells.

In [524]:
import numpy as np
import random
import copy
import time

# find the most restricted area in the graph, try a valid number
# repeat this process until the graph is complete

# therefore need a funciton which evaluates the restriction/completeness of a space,
# given its row, column and square data

# rank numbers given their frequency and assign a new number semi randomly

# check at each time step if the game has become invalid
# add the invalid state to a set, dont progress to invalid states


board = [["5","3",".",".","7",".",".",".","."],
         ["6",".",".","1","9","5",".",".","."],
         [".","9","8",".",".",".",".","6","."],
         ["8",".",".",".","6",".",".",".","3"],
         ["4",".",".","8",".","3",".",".","1"],
         ["7",".",".",".","2",".",".",".","6"],
         [".","6",".",".",".",".","2","8","."],
         [".",".",".","4","1","9",".",".","5"],
         [".",".",".",".","8",".",".","7","9"]]

board = np.array(board)

final_sol = 0
pos_sol = [np.copy(board)]
non_solutions = []

In [525]:
# returns the "flexibility" of a given point
def point_score(i, j, df):
    rv = np.count_nonzero(df[i, :] == ".")
    cv = np.count_nonzero(df[ :,j] == ".")
        
    box_i = np.argmax( np.nonzero([max(i+1 - 3*k, 0) for k in range(3)]))        
    box_j = np.argmax( np.nonzero([max(j+1 - 3*k, 0) for k in range(3)]))
        
    bv = np.count_nonzero(df[ 3*box_i:3+ 3*box_i ,3*box_j:3+ 3*box_j] == ".")
    
    return ((i, j), (box_i, box_j), 24 - rv - cv - bv)

def check_if_done(df):
    if np.count_nonzero(df == ".") == 0:
        print("We got em")
        final_sol = 1
        return True
    else:
        return False

# returns the most constrained points for a given board    
def cbps(df):
    sop = []
    mv = []

    for i in range(9):
        for j in range(9):
            if pos_sol[0][i, j] == ".":
        
                temp = point_score(i, j, pos_sol[0])
                sop.append((temp[0], temp[1], temp[2]))
                mv.append(temp[2])
    
    return [i for i in sop if i[2] == max(mv)]
    
    
lon = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]     
    
    
def add_num(point, box, c_board, lon):
    df = np.copy(c_board)
    
    lon = [i for i in lon if i not in df[point[0], :]] # remove all values constrained by row
    lon = [i for i in lon if i not in df[:,point[1]]] # remove all values constrained by column
    lon = [i for i in lon if i not in df[3*box[0]:3+3*box[0] ,3*box[1]:3+3*box[1]]]
    
    # if this board has valid children, create them and add them to the valid possible solutions
    #if len(lon) != 0: 
    for i in lon:
        df = np.copy(c_board)
        df[point[0], point[1]] = i
        pos_sol.append(df)
            
        
                   
    non_solutions.append(np.copy(c_board))
            

In [526]:
start = time.time()
while final_sol == 0:
    tl = len(pos_sol)
    tl2 = np.copy(tl)
    for j in range(tl2):
        c_board = pos_sol[j]
        c_points = cbps(c_board)
        
        for point in c_points:
            add_num(point[0], point[1], c_board, lon)
    
    # remove old boards
    pos_sol = [arr for arr in pos_sol if not any(np.array_equal(arr, sol) for sol in non_solutions)]
    
    # remove duplicate boards
    new_list = []
    for item in pos_sol:
        if not any(np.array_equal(item, arr) for arr in new_list):
            new_list.append(item)
    pos_sol = copy.deepcopy(new_list)
    
    for i in pos_sol:
        if check_if_done(i):
            final_sol = i
            break

    non_solutions = []
    
end  = time.time()
print(end - start)
print(final_sol)

We got em
1989.3275351524353
[['5' '3' '4' '6' '7' '8' '9' '1' '2']
 ['6' '7' '2' '1' '9' '5' '3' '4' '8']
 ['1' '9' '8' '3' '4' '2' '5' '6' '7']
 ['8' '5' '9' '7' '6' '1' '4' '2' '3']
 ['4' '2' '6' '8' '5' '3' '7' '9' '1']
 ['7' '1' '3' '9' '2' '4' '8' '5' '6']
 ['9' '6' '1' '5' '3' '7' '2' '8' '4']
 ['2' '8' '7' '4' '1' '9' '6' '3' '5']
 ['3' '4' '5' '2' '8' '6' '1' '7' '9']]


/var/folders/8g/j8v9v9b556b_06x_rbypfmh00000gn/T/ipykernel_64836/3420896407.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  while final_sol == 0:
